# Support Vector Machines in R

The R interface to libsvm in package e1071, svm(), was designed to be as intuitive as possible. Models are fitted and new data are predicted as usual, and both the vector/matrix and the formula interface are implemented. 

## Classification
In this example, we use the glass data from the UCI Repository of Machine Learning Databases for classification. The task is to predict the type of a glass on basis of its chemical analysis. We start by splitting the data into a train and
test set:

In [ ]:
install.packages("e1071")

In [4]:
library(e1071)

In [ ]:
install.packages("rpart")
library(rpart)

In [ ]:
install.packages("mlbench")
library(mlbench)
data(Glass, package="mlbench")
index     <- 1:nrow(Glass)
testindex <- sample(index, trunc(length(index)/3))
testset   <- Glass[testindex,]
trainset  <- Glass[-testindex,]

Both for the SVM and the partitioning tree (via rpart()), we fit the model and try to predict the test set values:

In [8]:
#svm
svm.model <- svm(Type ~ ., data = trainset, cost = 100, gamma = 1)
svm.pred  <- predict(svm.model, testset[,-10])

In [9]:
#rpart to show the regression results
rpart.model <- rpart(Type ~ ., data = trainset)
rpart.pred  <- predict(rpart.model, testset[,-10], type = "class")

In [10]:
## compute svm confusion matrix
table(pred = svm.pred, true = testset[,10])

    true
pred  1  2  3  5  6  7
   1 16  4  2  0  0  0
   2  6 18  2  4  1  4
   3  1  2  2  0  0  0
   5  0  0  0  0  0  0
   6  0  0  0  0  1  0
   7  0  0  0  0  0  8

In [11]:
## compute rpart confusion matrix
table(pred = rpart.pred, true = testset[,10])

    true
pred  1  2  3  5  6  7
   1 16  2  3  0  0  0
   2  6 17  1  0  2  1
   3  1  2  2  0  0  1
   5  0  2  0  3  0  0
   6  0  0  0  0  0  0
   7  0  1  0  1  0 10

Comparison between rart and svm
<img src="Selection_004.png">

## Regression
The regression capabilities of SVMs are demonstrated on the ozone data. Again, we split the data into a train and test set.

In [12]:
data(Ozone, package="mlbench")
index     <- 1:nrow(Ozone)
testindex <- sample(index, trunc(length(index)/3))
testset   <- na.omit(Ozone[testindex,-3])
trainset  <- na.omit(Ozone[-testindex,-3])

In [13]:
 ## svm
svm.model <- svm(V4 ~ ., data = trainset, cost = 1000, gamma = 0.0001)
svm.pred  <- predict(svm.model, testset[,-3])
crossprod(svm.pred - testset[,3]) / length(testindex)

11.1117


In [14]:
## rpart
rpart.model <- rpart(V4 ~ ., data = trainset)
rpart.pred  <- predict(rpart.model, testset[,-3])
crossprod(rpart.pred - testset[,3]) / length(testindex)

22.85684


Comparison between rpart and svm
<img src="Selection_003.png">

Drawbacks of using SVM:
1) SVMs scale rather badly with the data size due to the quadratic optimization algorithm and the kernel transformation. 
2) The correct choice of kernel parameters is crucial for obtaining good results, which practically means that an extensive search must be conducted on the parameter space before results can be trusted, and this often complicates the task (the authors of libsvm currently conduct some work on methods of efficient automatic parameter selection).  
3) The current implementation is optimized for the radial basis function kernel only, which clearly might be suboptimal for your data.